### Exercise 2 - Numerical Optimization

Consider the following nonlinear optimization problem

<img src="imgs/nlp.png" alt="nlp" width="350"/>


Compute the gradients of the objective $f = \frac{1}{2}(x-1)^2 + \frac{1}{2}(10(y - x^2))^2 + \frac{1}{2}x^2$ and the constraints $g = x + (1-y)^2 = 0$ and their Hessian on paper.
Write down on paper the Karush-Kuhn-Tucker (KKT) conditions for the above problem. Are these
conditions necessary for optimality? Are they sufficient?

Define CasADi functions for $f$ and $g$ and use CasADi to generate the gradient and hessian functions of these two functions.

In [ ]:
import numpy as np
from casadi import SX, vertcat

x = SX.sym('x')
y = SX.sym('y')

# FILL YOUR CODE HERE
# z = ...  # help: stack the variable in one vector for convenience

# f = ...  # declare cost function
# g = ...  # declare constraint function


Solve the above constrained optimization problem using CasADi using the solver `IPOPT`.

In [ ]:
from casadi import nlpsol

# FILL YOUR CODE HERE
solver = ...  # help: use casadi nlpsol, checkout is signature to understand what it needs
print(solver)

In [ ]:
# Call the solver to compute the optimal solution

# FILL YOUR CODE HERE
solution  = solver(...)  # checkout what have you print above to understand which arguments you have to pass to call "solver"
                         # NOTE: arguments are not mandatory and their default value is zero.

In [ ]:
print(f"{solution=}")

sol_opt = ...  # have a look to the overall solution and store the optimal primal values


In [ ]:
# BONUS PLOT! you should print it because it's really nice! ;)

import matplotlib.pyplot as plt
from matplotlib import ticker
from acados_template import latexify_plot
latexify_plot()

xlims = (-2, 2)
ylims = (-2, 2)
xs = np.linspace(xlims[0], xlims[1], 500)
ys = np.linspace(ylims[0], ylims[1], 500)
X, Y = np.meshgrid(xs, ys)

Z = 0.5*(X-1)**2 + 0.5*(10*(Y-X**2))**2 + 0.5*X**2

fig, ax = plt.subplots()
CS = ax.contourf(X, Y, Z, locator=ticker.LogLocator())
plt.colorbar(CS)

feasible_x = -(1-ys)**2

plt.plot(... , ... , color='w', label='feasible set') # TODO: fill out with the right argument to plot
plt.plot(... , ... , color='r', marker='o', label='optimum') # TODO: fill out with the right argument to plot
plt.legend()
plt.xlim(xlims)
plt.ylim(ylims)
plt.xlabel('$x$')
plt.ylabel('$y$');

Make sure that the KKT conditions are satisfied at the solution that you obtain.

In [ ]:
# Help: checkout the method get_function() of the function nlpsol -> solver.get_function()
# Understand which function you need to get to test the kkt conditions

nlp_grad = ...
print(nlp_grad)

In [ ]:
# call the function with the right arguments!

sol_opt = ...
lam_g_opt = ...
print(f"{nlp_grad(sol_opt, [], 1, lam_g_opt)=}")

### 2.4 Gentle Introduction to Numerical Optimal Control
We consider the dynamic system of Exercise 1 and now we aim to solve an optimization problem to compute the controls to apply to our system.
The inverted pendulum dynamics is given by:
\begin{equation}
\begin{aligned}
&\dot{\theta} =&& \omega \\
&\dot{\omega} =&& \sin(\theta) + \tau,
\end{aligned}
\end{equation}
where $\theta$ is the angle describing the orientation of the pendulum, $\omega$ is its angular velocity and $\tau$ is the input torque.

**We aim to compute a control trajectory that brings the pendulum in the upward position,** starting from $\bar{\theta} = -\pi$ and $\bar{\omega} = 0$.

<!--
    \begin{equation*}
    \begin{aligned}
    \underset{{{\mathbf{s}, \mathbf{a}}}}{\mathrm{min}} \; \textstyle   \sum_{k=0}^{N-1} &\;  l(s_k, a_k) + e(s_N) \\
    \textrm{s.t.} \quad 
    s_0 &= \bar{s}_0\\
    s_{k+1} &= f(s_k, a_k )  \\
    0 &\geq h(s_k, a_k), \; k = 0, \ldots, N\!-\!1\\
    0 &\geq r(s_N )
    \end{aligned}
    \end{equation*}
 -->

The NLP obtained via direct multiple shooting have the following form:

<img src="imgs/discrete-ocp.png" alt="discrete-ocp" width="350"/>



In order we will:
- Define the ODE
- Create a RK4 integrator to discretize the dynamics
- Define a quadratic stage cost for the optimal control problem
- Formulate a direct multiple shooting problem in CasADi
- Solve the problem with `ipopt`

In [ ]:
from casadi import SX, sin, Function, inf
import numpy as np

In [ ]:
# continuous model dynamics
n_s = 2  # number of states
n_a = 1  # number of actions

theta = SX.sym('theta')
omega = SX.sym('omega')
tau = SX.sym('tau')

# FILL IN YOUR CODE HERE
theta_dot = omega
omega_dot = sin(theta) + tau

s = vertcat(theta, omega)
s_dot = vertcat(theta_dot, omega_dot)

In [ ]:
def integrate_RK4(s_expr, a_expr, sdot_expr, dt, N_steps=1):
    '''RK4 integrator.

    s_expr, a_expr: casadi expression that have been used to define the dynamics sdot_expr
    sdot_expr:      casadi expr defining the rhs of the ode
    dt:             integration interval
    N_steps:        number of integration steps per integration interval, default:1
    '''

    h = dt/N_steps

    s_end = s_expr

    sdot_fun = Function('xdot', [s_expr, a_expr], [sdot_expr])

    for _ in range(N_steps):

    # FILL IN YOUR CODE HERE
        v_1 = sdot_fun(s_end, a_expr)
        v_2 = sdot_fun(s_end + 0.5 * h * v_1, a_expr)
        v_3 = sdot_fun(s_end + 0.5 * h * v_2, a_expr)
        v_4 = sdot_fun(s_end + v_3 * h, a_expr)
        s_end += (1/6) * (v_1 + 2 * v_2 + 2 * v_3 + v_4) * h

    F_expr = s_end

    return F_expr

The stage cost should have the following form
\begin{equation}
    l(x, u) = \frac{1}{2} \left( x^\top Q x + u^\top R u \right)
\end{equation}
and the terminal cost
\begin{equation}
    e(x) = \frac{1}{2} \left( x^\top Q x \right)
\end{equation}
with
\begin{equation}
    Q = \begin{bmatrix} 10 & 0 \\ 0 & 1 \end{bmatrix}
\end{equation}
and
\begin{equation}
    R = \begin{bmatrix} 1 \end{bmatrix}
\end{equation}
**Discretize the stage cost with explicit Euler:** $\int_{t_i}^{t_{i+1}}l(x(t), u(t))dt \approx (t_{i+1} - t_i) \cdot l(x(t_i), u(t_{i}))$.

In [ ]:
# Define number of steps in the control horizon and discretization step
N = 200
delta_t = 1/20
# Define RK4 integrator function and initial state x0_bar
F_rk4 = Function(...)  # TODO: FILL WITH YOUR CODE
x0_bar = [...]  # TODO: FILL WITH YOUR CODE

# Define the weighting matrix for the cost function
Q = ...  # TODO: FILL WITH YOUR CODE
R = ...  # TODO: FILL WITH YOUR CODE

We want to constrain the action such that $\tau \in [-1, 1]$ and state such that $\theta \in [-\pi/2, 2\pi]$ while $\omega$ is free.

To enforce constraint on the optimization variables directly (as box constraint) we can use CasADi `lbw, ubw`.

*Hint 1: enforce the initial state by setting the* `lbw, ubw` *corresponding to the $s_0$ variable to $\bar{s}_0$.*

For enforcing generic constraint, we first define them as CasADi expression in `g` and then assign the respective lower and upper bound via `lbg, ubg`. 

*Hint 2: this is what we need to do for defining the multiple shooting constraints.*



In [ ]:
# Start with an empty NLP
w = []
w0 = []
lbw = []
ubw = []
J = 0
g = []
lbg = []
ubg = []

# "Lift" initial conditions
Xk = SX.sym(...)  # TODO: FILL WITH YOUR CODE
w += [Xk]
lbw += ...      # TODO: FILL WITH YOUR CODE
ubw += ...      # TODO: FILL WITH YOUR CODE
w0 += ...       # TODO: FILL WITH YOUR CODE

# Formulate the NLP
for k in range(N):
    # New NLP variable for the control
    Uk = SX.sym('U_' + str(k))
    w   += ...   # TODO: FILL WITH YOUR CODE
    lbw += ...   # TODO: FILL WITH YOUR CODE
    ubw += ...   # TODO: FILL WITH YOUR CODE
    w0  += ...   # TODO: FILL WITH YOUR CODE

    # Integrate till the end of the interval
    Xk_end = F_rk4(Xk, Uk)
    J = J + ...  # TODO: FILL WITH YOUR CODE - Complete with the stage cost

    # New NLP variable for state at end of interval
    Xk = SX.sym(...)  # TODO: FILL WITH YOUR CODE
    w   += ...  # TODO: FILL WITH YOUR CODE
    lbw += ...  # TODO: FILL WITH YOUR CODE
    ubw += ...  # TODO: FILL WITH YOUR CODE
    w0  += ...  # TODO: FILL WITH YOUR CODE

    # Add equality constraint to "close the gap" for multiple shooting
    g   += ...  # TODO: FILL WITH YOUR CODE
    lbg += ...
    ubg += ...
J = J + ...  # TODO: FILL WITH YOUR CODE: Complete with the terminal cost (NOTE it should be weighted by delta_t)

# Create an NLP solver
prob = {'f': J, 'x': vertcat(*w), 'g': vertcat(*g)}
solver = nlpsol('solver', 'ipopt', prob)

# Solve the NLP
sol = solver(x0=w0, lbx=lbw, ubx=ubw, lbg=lbg, ubg=ubg)
w_opt = sol['x'].full().flatten()

Retrieve the state and action trajectories and plot them.

In [ ]:
from plot_utils import plot_trajectories
s_traj_opt = np.vstack((w_opt[::3], w_opt[1::3]))
a_traj_opt = w_opt[2::3][np.newaxis, ...]
plot_trajectories(s_traj_opt, a_traj_opt, labels='optimal')
